This notebook demonstrates the use of Robert Kern's `line_profiler` (https://github.com/rkern/line_profiler) to profile `cython` functions line by line. Thanks to Robert Bradshaw for implementing this feature (https://groups.google.com/forum/#!topic/cython-users/FH3TYK8BkeA).

In [1]:
#Load Robert Kern's line profiler
%load_ext line_profiler
%load_ext Cython
import line_profiler

In [2]:
#Set compiler directives (cf. http://docs.cython.org/src/reference/compilation.html)
import Cython
directive_defaults = Cython.Compiler.Options.get_directive_defaults()

directive_defaults['linetrace'] = True
directive_defaults['binding'] = True

In [3]:
import numpy as np

In [4]:
%%cython -a -f --compile-args=-DCYTHON_TRACE=1
#We need to define the macro CYTHON_TRACE=1 (cf. http://docs.cython.org/src/reference/compilation.html)

cimport numpy as np
np.import_array()

def centroid_inertia(np.ndarray c, np.ndarray d, np.ndarray l, int k):

    cdef np.ndarray[np.float_t, ndim=2] centroids = c
    cdef np.ndarray[np.float_t, ndim=2] data = d
    cdef np.ndarray[np.int_t, ndim=1] labels = l

    cdef float total, point_dist
    cdef int i, val, pt_idx
    cdef np.ndarray[np.float_t, ndim=1] pt
    cdef np.ndarray[np.float_t, ndim=1] centroid

    for i in range(k):
        centroid = centroids[i]

        for pt_idx in range(len(data)):
            if labels[pt_idx] != i:
                continue
                
            pt = data[pt_idx]
            point_dist = 0
            for val in range(len(pt)):
                point_dist += (centroid[val] - pt[val]) ** 2
            total += (point_dist ** .5)

    return total

In [6]:
#Print profiling statistics using the `line_profiler` API
profile = line_profiler.LineProfiler(centroid_inertia)
c = np.array([i for i in range(6)], dtype='float').reshape((3,2))
d = np.array([i for i in range(20)], dtype='float').reshape((10,2))
l = np.array([0,2,1,0,2,1,2,2,1,0], dtype='int')
profile.runcall(centroid_inertia, c, d, l, 3)
profile.print_stats()

Timer unit: 1e-06 s

Total time: 7.7e-05 s
File: /Users/cbitting/.ipython/cython/_cython_magic_0d02272fb11a5b3ab322e901b8a22787.pyx
Function: centroid_inertia at line 6

Line #      Hits         Time  Per Hit   % Time  Line Contents
     6                                           def centroid_inertia(np.ndarray c, np.ndarray d, np.ndarray l, int k):
     7                                           
     8         1         10.0     10.0     13.0      cdef np.ndarray[np.float_t, ndim=2] centroids = c
     9         1          1.0      1.0      1.3      cdef np.ndarray[np.float_t, ndim=2] data = d
    10         1          1.0      1.0      1.3      cdef np.ndarray[np.int_t, ndim=1] labels = l
    11                                           
    12                                               cdef float total, point_dist
    13                                               cdef int i, val, pt_idx
    14                                               cdef np.ndarray[np.float_t, ndim=1] 

In [7]:
#Print profiling statistics using the `lprun` magic
%lprun -f centroid_inertia centroid_inertia(c, d, l, 3)